## 包导入

In [1]:
import os
import json
import time
import pymysql
import warnings
import requests

import numpy as np
import pandas as pd
import pandas_profiling

import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line,Pie,Map,Timeline,Tab

D:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## 数据采集

### 中职教育数据总体采集

In [1]:
# 查看浏览器代理：about://version
agent={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
basepath=r'C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据'
# 子集文件夹自动化创建【保证 国家统计局(csv)文件夹 必须存在】
if not os.path.exists(basepath):
    os.mkdir(basepath)
# 忽略warning警告错误
warnings.filterwarnings('ignore')

baseurl='https://data.stats.gov.cn/easyquery.htm'
#如果是国家季度数据，dbcode（database-code）设为'hgjd',如果是月度数据,dbcode设为'hgyd','wdcode'等均不变
dbcode,wdcode,rowcode,colcode='hgnd','zb','zb','sj'

def retry(url,params,headers):
    while True:
        try:
            req=requests.get(url,params=params,headers=headers,verify=False)
            req.encoding=req.apparent_encoding
            trans_dict=json.loads(req.text) # json.loads()读取字符串.json.load()读取文件
            break
        except:
            time.sleep(2)
            print('服务器繁忙')
    return trans_dict

#定义爬取函数
def packcsv(valuecode,path):
    base_can={'m':'QueryData','dbcode':dbcode,'rowcode':rowcode,'colcode':colcode,
              'wds':'[]','dfwds':'[{"wdcode":"zb","valuecode":"%s"},{"wdcode":"sj","valuecode":"2000-2022"}]'%valuecode,
              'k1':str(int(time.time()*1000))}
    trans_dict=retry(baseurl,base_can,agent) # trans_dict'type:[{}]
    
    # 数据在strdata中，但是strdata被层层包裹：遍历列表通过键获取值
    #数据查探：returndata->list:datanodes->dict:datanodes[0]->data->dict(strdata:'103.8')
    row_loc=trans_dict['returndata']['wdnodes'][0]['nodes'] # row_loc'type:{...[0:{}]}
    totaldata=trans_dict['returndata']['datanodes']
    frame_dict={each['code']:pd.DataFrame(columns=['年份',each['cname']]) for each in row_loc}#DF转存表格格式
    
    # 遍历数据集
    for i in totaldata: 
        match_year=int(i['wds'][1]['valuecode'])#获取整型年份
        match_code=i['wds'][0]['valuecode']#获取字段编码
        match_row=list(frame_dict[match_code].columns)[-1]#通过frame_dict对应的DF列名获取字段中文名称
        if i['data']['hasdata']:
            match_data=i['data']['data']
        else:
            match_data=float('nan')#有数据则取值，无则nan
        eachcsv=frame_dict[match_code]
        eachcsv=eachcsv.append({'年份':match_year,match_row:match_data},ignore_index=True)
        frame_dict[match_code]=eachcsv#对字典中每一个字段名对应的frame进行逐行append操作
        
    #frame_dict生成完毕，循环merge成一个大的dataframe
    for i in frame_dict:
        try:
            csv=pd.merge(csv,frame_dict[i],how='outer',on=['年份'])#pd.merge:对散数据进行拼接
        except NameError:
            csv=frame_dict[i]#csv未定义，则以初始的frame_dict中的frame作为初始值进行merge
    csv['年份']=csv['年份'].astype("int64")#生成的csv文件年份数据从浮点数转为int
    csv.to_csv(path,index=False)#保存文件
code_path={'zb':basepath}
while code_path:
    each_root={}
    for each in code_path:
        if each=='zb' or ('A0M' in each)==True:
            add_can={'id':each,'dbcode':dbcode,'wdcode':wdcode,'m':'getTree'}
            root_req=retry(baseurl,add_can,agent)
            for i in root_req:
                #print(i)
                #如果有一些中文名带单位的，这种斜杠会对python识别路径产生干扰
                rootname=i['name'].replace('/','每')
                idsign=i['id']
                makedir=code_path[each]+'\\'+rootname
                if i['isParent'] and ('A0M' in i['id'])==True:
                    if not os.path.exists(makedir):
                        os.mkdir(makedir)
                        print('已创建路径%s'%makedir)
                    else:
                        #此前调试使用，如果路径存在，则不用创建路径，直接将路径保存到each_root即可，这样调试不对就不用每次删除文件夹了
                        print('已有路径%s'%makedir)
                    each_root[idsign]=makedir
                elif ('A0M' in i['id'])==True:
                    print('正在写表,对应路径为%s'%(makedir+'.csv'))
                    packcsv(idsign,makedir+'.csv')
                    print('写表完成')
    code_path=each_root

已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育
已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校、教职工和专任教师情况
已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学历教育学生情况
已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类非学历教育学生情况
已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类民办教育基本情况
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校专任教师数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校招生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校在校学生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\各级各类学校毕业生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\研究生和留学生数.csv
写表完成
已创建路径C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局

写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\普通高中学校和学生情况\普通高中招生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\普通高中学校和学生情况\普通高中在校学生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\普通高中学校和学生情况\普通高中毕业生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\初中学校和学生情况\普通初中学校数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\初中学校和学生情况\职业初中学校数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\初中学校和学生情况\初中招生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\初中学校和学生情况\初中在校学生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\初中学校和学生情况\初中毕业生数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\普通小学学校和学生情况\普通小学学校数.csv
写表完成
正在写表,对应路径为C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局年度数据\教育\普通小学学校和学生情况\普通小学招生数.csv


### 中职教育数据分省采集

In [2]:
agent={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
basepath='C:\\Users\\1700\\Desktop\\职业教育分析\\Data\\RawData\\国家统计局分省年度数据'
if not os.path.exists(basepath):
    os.mkdir(basepath)
warnings.filterwarnings('ignore')#对应后文verify=false,这样就不用每次运行都跳出来一个warning
baseurl='https://data.stats.gov.cn/easyquery.htm'
dbcode,wdcode,rowcode,colcode='fsnd','zb','zb','sj'
def retry(url,params,headers):
    while True:
        try:
            req=requests.get(url,params=params,headers=headers,verify=False)
            req.encoding=req.apparent_encoding
            trans_dict=json.loads(req.text)
            break
        except:#请求的html网页均为json型，如果解析出错，则为请求过多暂未相应，等待并持续请求即可
            time.sleep(2)
            print('服务器繁忙')
    return trans_dict
#定义爬取函数
def packcsv(valuecode,path):
    zhibiao_info={'m':'getOtherWds','dbcode':dbcode,'rowcode':'reg','colcode':colcode,
                  'wds':'[{"wdcode":"zb","valuecode":"%s"}]'%valuecode,'dfwds':'[]',
                  'k1':str(int(time.time()*1000))}
    trans_dict=retry(baseurl,zhibiao_info,agent)
    choose_range=trans_dict['returndata'][0]['nodes']
    matchdict={each['code']:each['name'] for each in choose_range}
    count_time=0
    for each in matchdict:
        zb_name=matchdict[each]
        each_csv=pd.DataFrame(columns=['省份编码','省份','年份',zb_name])
        base_can={'m':'QueryData','dbcode':dbcode,'rowcode':'reg','colcode':colcode,
                  'wds':'[{"wdcode":"zb","valuecode":"%s"}]'%each,
                  'dfwds':'[{"wdcode":"zb","valuecode":"%s"},{"wdcode":"sj","valuecode":"2000-2019"}]'%each,
                  'k1':str(int(time.time()*1000))}
        trans_dict=retry(baseurl,base_can,agent)
        province_code=trans_dict['returndata']['wdnodes'][1]['nodes']
        province_dict={each['code']:each['cname'] for each in province_code}
        totaldata=trans_dict['returndata']['datanodes']
        for i in totaldata:
            match_year=int(i['wds'][2]['valuecode'])
            match_code=i['wds'][1]['valuecode']
            match_province=province_dict[match_code]
            if i['data']['hasdata']:
                match_data=i['data']['data']
            else:
                match_data=float('nan')
            each_csv=each_csv.append({'省份编码':match_code,'省份':match_province,
                                      '年份':match_year,zb_name:match_data},ignore_index=True)
        try:
            csv=pd.merge(csv,each_csv,on=['省份','年份','省份编码'],how='outer')
        except NameError:
            csv=each_csv
        count_time+=1
        print('写表完成%.2f%%'%(100*count_time/len(matchdict)))
    csv.to_csv(path,index=False)
code_path={'zb':basepath}
while code_path:
    each_root={}
    for each in code_path:
        if each=='zb' or ('A0M' in each)==True:
            add_can={'id':each,'dbcode':dbcode,'wdcode':wdcode,'m':'getTree'}
            root_req=retry(baseurl,add_can,agent)
            for i in root_req:
                rootname=i['name'].replace('/','每')
                idsign=i['id']
                makedir=code_path[each]+'\\'+rootname
                if i['isParent'] and ('A0M' in i['id'])==True:
                    if not os.path.exists(makedir):
                        os.mkdir(makedir)
                        print('已创建路径%s'%makedir)
                    else:
                        print('已有路径%s'%makedir)
                    each_root[idsign]=makedir
                elif ('A0M' in i['id'])==True:
                    print('正在写表,对应路径为%s'%(makedir+'.csv'))
                    packcsv(idsign,makedir+'.csv')
                    print('写表完成')
    code_path=each_root

已创建路径C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育
正在写表,对应路径为C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育\高等学校普通本、专科学校和学生情况.csv
写表完成7.14%
写表完成14.29%
写表完成21.43%
写表完成28.57%
写表完成35.71%
写表完成42.86%
写表完成50.00%
写表完成57.14%
写表完成64.29%
写表完成71.43%
写表完成78.57%
写表完成85.71%
写表完成92.86%
写表完成100.00%
写表完成
正在写表,对应路径为C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育\普通高等学校(机构)教职工情况.csv
写表完成9.09%
写表完成18.18%
写表完成27.27%
写表完成36.36%
写表完成45.45%
写表完成54.55%
写表完成63.64%
写表完成72.73%
写表完成81.82%
写表完成90.91%
写表完成100.00%
写表完成
正在写表,对应路径为C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育\中等职业学校(机构)学生情况.csv
写表完成14.29%
写表完成28.57%
写表完成42.86%
写表完成57.14%
写表完成71.43%
写表完成85.71%
写表完成100.00%
写表完成
正在写表,对应路径为C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育\中等职业学校(机构)教职工数.csv
写表完成11.11%
写表完成22.22%
写表完成33.33%
写表完成44.44%
写表完成55.56%
写表完成66.67%
写表完成77.78%
写表完成88.89%
写表完成100.00%
写表完成
正在写表,对应路径为C:\Users\1700\Desktop\职业教育分析\Data\RawData\国家统计局分省年度数据\教育\普通高中基本情况.csv
写表完成16.67%
写表完成33.33%
写表完成50.00%
写表完成

## 数据清洗

### 本地数据导出

In [2]:
def show_files_path(path, all_files_path):
    # 显示当前目录所有文件和子文件夹，放入file_list数组里
    file_list = os.listdir(path)
    # 循环判断每个file_list里的元素是文件夹还是文件，是文件的话，把名称传入list.
    for file in file_list:
        if "中等职业" in file:
             # 利用os.path.join()方法取得路径全名，并存入tmp变量
            tmp = os.path.join(path, file)
            if os.path.isdir(tmp)==False:
                all_files_path.append(os.path.join(path, file))
    return all_files_path

### 总体类数据清洗

In [3]:
path1 = show_files_path("C:/Users/1700/Desktop/职业教育分析/Data/RawData/国家统计局年度数据/教育/", [])

# 中等职业学校(机构)学生情况.csv：数据清洗
data1 = pd.read_csv(path1[0])
data1 = data1.iloc[[i for i in range(1,14)],[0,1,4,5,6,7]].sort_index(ascending=False).reset_index(drop=True)
data1 = data1.rename(columns=(lambda x:x[10:])).rename(columns={'':'年份'})
#print('中等职业学校(机构)学生情况.csv\n',data1)

# 中等职业学校(机构)教职工数.csv：数据清洗
data2 = pd.read_csv(path1[1])
data2 = data2.iloc[[i for i in range(1,14)],[0,1,3]].sort_index(ascending=False).reset_index(drop=True)
data2 = data2.rename(columns=(lambda x:x[10:])).rename(columns={'':'年份'})
#print('中等职业学校(机构)教职工数.csv\n',data2)

# 中等职业学校(机构)数.csv：数据清洗
data3 = pd.read_csv(path1[2])
data3 = data3.iloc[[i for i in range(1,14)],[0,2,3]].sort_index(ascending=False).reset_index(drop=True)
print('中等职业学校(机构)数.csv\n',data3)

D:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


中等职业学校(机构)数.csv
       年份  普通中等专业学校数  成人中等专业学校数
0   2008     3846.0     1983.0
1   2009     3789.0     1883.0
2   2010     3938.0     1720.0
3   2011     3753.0     1614.0
4   2012     3681.0     1564.0
5   2013     3577.0     1536.0
6   2014     3536.0     1457.0
7   2015     3456.0     1294.0
8   2016     3398.0     1243.0
9   2017     3346.0     1218.0
10  2018     3322.0     1097.0
11  2019     3339.0     1032.0
12  2020     3266.0      991.0


### 分科类数据清洗

In [3]:
path3 = show_files_path("C:/Users/1700/Desktop/职业教育分析/Data/RawData/国家统计局年度数据/教育/中等职业学校(机构)学生分科类情况/", [])

# 中等职业学校(机构)学生分科类在校学生数.csv：数据清洗
data4 = pd.read_csv(path3[0]).drop('中等职业学校(机构)在校学生数',axis=1)
data4 = data4.iloc[[i for i in range(6,12)]].sort_index(ascending=False).reset_index(drop=True)
data4 = data4.rename(columns=(lambda x:x.replace("在校学生数", "").replace("中等职业学校(机构)","")))
#print('中等职业学校(机构)学生分科类在校学生数.csv\n',data4)

# 中等职业学校(机构)学生分科类招生数.csv：数据清洗
data5 = pd.read_csv(path3[1]).drop('中等职业学校(机构)招生数',axis=1)
data5 = data5.iloc[[i for i in range(6,12)]].sort_index(ascending=False).reset_index(drop=True)
data5 = data5.rename(columns=(lambda x:x.replace("招生数", "").replace("中等职业学校(机构)","")))
#print('中等职业学校(机构)学生分科类招生数.csv\n',data5)

# 中等职业学校(机构)学生分科类毕业生数.csv：数据清洗
data6 = pd.read_csv(path3[2]).drop('中等职业学校(机构)毕业生数',axis=1)
data6 = data6.iloc[[i for i in range(6,12)]].sort_index(ascending=False).reset_index(drop=True)
data6 = data6.rename(columns=(lambda x:x.replace("毕业生数", "").replace("中等职业学校(机构)","")))
#print('中等职业学校(机构)学生分科类毕业生数.csv\n',data6)

# 中等职业学校(机构)学生分科类获得职业资格证书毕业生数.csv：数据清洗
data7 = pd.read_csv(path3[3]).drop('中等职业学校(机构)获得职业资格证书毕业生数',axis=1)
data7 = data7.iloc[[i for i in range(6,12)]].sort_index(ascending=False).reset_index(drop=True)
data7 = data7.rename(columns=(lambda x:x.replace("获得职业资格证书毕业生数", "").replace("中等职业学校(机构)","")))
print('中等职业学校(机构)学生分科类获得职业资格证书毕业生数.csv\n',data7)

中等职业学校(机构)学生分科类获得职业资格证书毕业生数.csv
      年份    农林牧渔类  资源与环境类  能源与新能源类  土木水利工程类    加工制造类    交通运输类    信息技术类  \
0  2010  12.4565  2.7987   2.2830   9.0821  92.5115  15.2335  87.0967   
1  2011  18.4656  2.2605   2.2050   9.2935  83.3267  16.9349  83.1043   
2  2012  28.1080  2.3394   1.9102  10.3482  70.0805  21.9221  78.5308   
3  2013  54.4841  3.5561   2.3213  14.0428  77.3784  27.3548  84.0227   
4  2014  44.9566  2.7962   2.0262  16.7524  69.7743  27.6981  74.2863   
5  2015  39.6586  2.8599   1.7162  16.1698  60.8863  29.8141  67.3675   

     医药卫生类   休闲保健类    财经商贸类    旅游服务类    文化艺术类   体育与健身      教育类   司法服务类  \
0  16.1472  0.6716  39.2009  12.8738  17.3439  1.9521  13.2573  1.1079   
1  17.9303  0.7313  40.1571  14.4294  15.1098  2.1261  15.7587  1.1330   
2  23.1860  1.2857  39.3229  16.4360  15.0245  1.7406  19.8340  1.0807   
3  30.9022  2.0132  48.0596  18.7459  18.9396  2.3140  29.5739  1.5887   
4  28.4019  1.8905  42.7803  17.3236  18.0086  2.1961  37.0190  1.5854   
5  31.3538 

### 分省类数据清洗

In [5]:
path2 = show_files_path("C:/Users/1700/Desktop/职业教育分析/Data/RawData/国家统计局分省年度数据/教育/", [])

# 分省:中等职业学校(机构)学生情况.csv：数据清洗
data8 = pd.read_csv(path2[0])
data8 = data8.loc[data8['年份'].isin([i for i in range(2008,2020)])].iloc[:,[1,2,3,6,7,8,9]].sort_index(ascending=False).reset_index(drop=True)
data8 = data8.rename(columns=(lambda x:x.replace("中等职业学校","")))
print(data8.head(50))

# 分省:中等职业学校(机构)教职工数.csv：数据清洗
data9 = pd.read_csv(path2[1])
data9 = data9.loc[data9['年份'].isin([i for i in range(2008,2020)])].iloc[:,[1,2,3,5]].sort_index(ascending=False).reset_index(drop=True)
data9 = data9.rename(columns=(lambda x:x.replace("中等职业学校(机构)","")))
#print(data9.head(50))

          省份    年份  招生数(万人)  在校学生数(万人)  毕业生数(万人)  获得职业资格证书毕业生数(万人)  预计毕业生数(万人)
0   新疆维吾尔自治区  2008   9.1979    21.1001    4.2831            1.9845      6.3134
1   新疆维吾尔自治区  2009   8.8484    21.7555    5.9414            2.9639      6.5806
2   新疆维吾尔自治区  2010   9.8894    23.3319    6.6383            3.4953      6.9700
3   新疆维吾尔自治区  2011   9.0177    24.3039    6.4508            3.8080      7.2113
4   新疆维吾尔自治区  2012   8.6751    23.5277    7.1288            4.0546      7.7522
5   新疆维吾尔自治区  2013   8.3893    22.9294    7.4860            5.0888      7.5347
6   新疆维吾尔自治区  2014   8.5937    21.9483    7.0764            5.5338      7.1150
7   新疆维吾尔自治区  2015   8.9493    22.1705    6.5161            4.8583      6.1491
8   新疆维吾尔自治区  2016   8.8939    23.5108    5.4934            4.2590      6.9736
9   新疆维吾尔自治区  2017   8.5511    23.8694    6.4318            4.7649      7.2343
10  新疆维吾尔自治区  2018  10.0792    25.3657    6.8483            4.7120      7.3226
11  新疆维吾尔自治区  2019   8.7100    25.5437    6.7961    

## 数据集成

### 中职教育发展现状总体数据

In [38]:
Data0101 = pd.merge(data1,data2,how='left')
Data01 = pd.merge(Data0101,data3,how='left')
# Data01 = Data01.set_index("年份")
Data01.to_csv(r'C:\Users\1700\Desktop\职业教育分析\Data\CookedData\GeneralData.csv')

### 中职教育发展现状分省数据

In [35]:
Data02 = pd.merge(data8,data9,how='left')
# Data02 = Data02.set_index("年份")
Data02.to_csv(r'C:\Users\1700\Desktop\职业教育分析\Data\CookedData\ProvincialData.csv')

### 数据库存储

In [41]:
pd.set_option('expand_frame_repr', False)#True就是可以换行显示。设置成False的时候不允许换行
pd.set_option('display.max_columns', None)# 显示所有列
#pd.set_option('display.max_rows', None)# 显示所有行
pd.set_option('colheader_justify', 'centre')# 显示居中

try:
    conn = pymysql.connect(host='localhost', user='root', password='123456', db='svedu', charset='utf8')
    cur = conn.cursor()
    print('数据库连接成功！')
    print(' ')
except:
    print('数据库连接失败！')

def gomysql(f,filename):
    columns = f.columns.tolist()  #获取表格数据内的列标题文字数据
    types = f.dtypes  #获取文件内数据格式
    field = []  #设置列表用来接收文件转换后的数据，为写入mysql做准备
    table = []
    char = []

    for item in range(len(columns)):  #开始循环获取文件格式类型并将其转换成mysql文件格式类型
        if 'object' == str(types[item]):
            char = '`' + columns[item] + '`' + ' VARCHAR(255)'  #必须加上`这个点，否则在写入mysql是会报错
        elif 'int64' == str(types[item]):
            char = '`' + columns[item] + '`' + ' INT'
        elif 'float64' == str(types[item]):
            char = '`' + columns[item] + '`' + ' FLOAT'
        elif 'datetime64[ns]' == str(types[item]):
            char = '`' + columns[item] + '`' + ' DATETIME'
        else:
            char = '`' + columns[item] + '`' + ' VARCHAR(255)'
        table.append(char)
        field.append('`' + columns[item] + '`')

    tables = ','.join(table)  #将table中的元素用，连接起来为后面写入mysql做准备
    fields = ','.join(field)

    cur.execute('drop table if exists {};'.format(filename))
    conn.commit()

    #创建表格并设置表格的列文字跟累数据格式类型
    table_sql = 'CREATE TABLE IF NOT EXISTS ' + filename + '(' + 'id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,' + tables + ');'
    print('表:' + filename + ',开始创建数据表...')
    cur.execute(table_sql)
    conn.commit()
    print('表:' + filename + ',创建成功!')

    print('表:' + filename + ',正在写入数据当中...')
    f_sql = f.astype(object).where(pd.notnull(f), None)  #将原来从csv文件获取得到的空值数据设置成None，不设置将会报错
    values = f_sql.values.tolist()  #获取数值
    s = ','.join(['%s' for _ in range(len(f.columns))])  #获得文件数据有多少列，每个列用一个 %s 替代
    insert_sql = 'insert into {}({}) values({})'.format(filename,fields,s)
    cur.executemany(insert_sql, values)
    conn.commit()
    print('表:' + filename + ',数据写入完成！')
    print(' ')
    cur.close()
    conn.close()
    print('文件导入数据库完成!')
    
# gomysql(Data01,'GeneralData')
gomysql(Data02,'ProvincialData')

数据库连接成功！
 
表:ProvincialData,开始创建数据表...
表:ProvincialData,创建成功!
表:ProvincialData,正在写入数据当中...
表:ProvincialData,数据写入完成！
 
文件导入数据库完成!


## 数据分析

### 数据库数据导出

In [4]:
try:
    conn = pymysql.connect(host='localhost', user='root', password='123456', db='svedu', charset='utf8')
    cur = conn.cursor()
    print('数据库连接成功！')
    print(' ')
except:
    print('数据库连接失败！')
    
sql1 = '''select * from generaldata'''
sql2 = '''select * from provincialdata'''
data1 = pd.read_sql(sql1,conn)
data2 = pd.read_sql(sql2,conn)
print(data1.head())
data2.head()

数据库连接成功！
 
   id    年份      招生数    在校学生数     毕业生数  获得职业资格证书毕业生数   预计毕业生数    教职工总数  \
0   1  2008  650.274  1688.24  471.092       281.926  521.847  97.3431   
1   2  2009  711.777  1779.85  509.665       313.126  561.003  96.9477   
2   3  2010  711.396  1816.44  543.652       342.032  563.213  95.6631   
3   4  2011  649.963  1774.91  541.125       338.429  573.474  94.5081   
4   5  2012  597.078  1689.88  554.384       348.387  566.395  92.1332   

     专任教师数  普通中等专业学校数  成人中等专业学校数  
0  67.4169     3846.0     1983.0  
1  68.2151     3789.0     1883.0  
2  68.0954     3938.0     1720.0  
3  68.9363     3753.0     1614.0  
4  68.4071     3681.0     1564.0  


,id,省份,年份,招生数(万人),在校学生数(万人),毕业生数(万人),获得职业资格证书毕业生数(万人),预计毕业生数(万人),教职工总数(万人),专任教师数(万人)
0,1,新疆维吾尔自治区,2008,9.1979,21.1001,4.2831,1.9845,6.3134,1.4741,0.9933
1,2,新疆维吾尔自治区,2009,8.8484,21.7555,5.9414,2.9639,6.5806,1.5334,1.0504
2,3,新疆维吾尔自治区,2010,9.8894,23.3319,6.6383,3.4953,6.9700,1.4988,1.0084
3,4,新疆维吾尔自治区,2011,9.0177,24.3039,6.4508,3.8080,7.2113,1.4936,1.0382
4,5,新疆维吾尔自治区,2012,8.6751,23.5277,7.1288,4.0546,7.7522,1.4739,1.0352


### 中职教育发展总体性分析

#### 描述性统计分析

In [3]:
data1.shape

(13, 11)

In [4]:
data1.columns

Index(['id', '年份', '招生数', '在校学生数', '毕业生数', '获得职业资格证书毕业生数', '预计毕业生数', '教职工总数',
       '专任教师数', '普通中等专业学校数', '成人中等专业学校数'],
      dtype='object')

In [5]:
data1.describe()

,id,年份,招生数,在校学生数,毕业生数,获得职业资格证书毕业生数,预计毕业生数,教职工总数,专任教师数,普通中等专业学校数,成人中等专业学校数
count,13.00000,13.00000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,7.00000,2014.00000,548.085231,1481.926154,476.102538,335.420077,483.631154,87.656400,66.196985,3557.461538,1433.230769
std,3.89444,3.89444,102.780143,238.715123,65.735045,50.364367,74.066646,6.875578,1.911462,224.456609,318.745236
min,1.00000,2008.00000,428.502000,1213.630000,383.464000,257.912000,385.137000,79.959300,63.546100,3266.000000,991.000000
25%,4.00000,2011.00000,466.143000,1267.840000,406.398000,299.774000,413.698000,81.114700,64.314300,3346.000000,1218.000000
50%,7.00000,2014.00000,495.355000,1416.310000,473.265000,338.429000,490.288000,86.690500,66.378200,3536.000000,1457.000000
75%,10.00000,2017.00000,649.963000,1689.880000,541.125000,354.767000,561.003000,94.508100,68.095400,3753.000000,1614.000000
max,13.00000,2020.00000,711.777000,1816.440000,557.559000,433.694000,573.474000,97.343100,68.936300,3938.000000,1983.000000


In [6]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            13 non-null     int64  
 1   年份            13 non-null     int64  
 2   招生数           13 non-null     float64
 3   在校学生数         13 non-null     float64
 4   毕业生数          13 non-null     float64
 5   获得职业资格证书毕业生数  13 non-null     float64
 6   预计毕业生数        13 non-null     float64
 7   教职工总数         13 non-null     float64
 8   专任教师数         13 non-null     float64
 9   普通中等专业学校数     13 non-null     float64
 10  成人中等专业学校数     13 non-null     float64
dtypes: float64(9), int64(2)
memory usage: 1.2 KB


#### 特征关联性分析

In [10]:
data11 = pd.DataFrame(data1,columns=['年份','招生数','在校学生数', '毕业生数',
                            '获得职业资格证书毕业生数','教职工总数','普通中等专业学校数'])
data11.rename(columns={'年份':'Year',
                                '招生数':'Enrollment',
                                '在校学生数':'Population',
                                '毕业生数':'Graduation',
                                '获得职业资格证书毕业生数':'VQGraduation',
                                '教职工总数':'Staff',
                                '普通中等专业学校数':'School',
                               },inplace=True)

a = pandas_profiling.ProfileReport(data11, title='GeneralData analysis')
a.to_file("AnalysisReport.html")

Summarize dataset:   0%|          | 0/20 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#### 关联特征对照分析

In [6]:
x = [str(i)+'年' for i in data11['Year'].tolist()]
y1 = data11['Enrollment'].tolist()
y2 = data11['Population'].tolist()
y3 = data11['Graduation'].tolist()
y4 = data11['VQGraduation'].tolist()
y5 = data11['Staff'].tolist()
y6 = data11['School'].tolist()

c = (
    Line(init_opts=opts.InitOpts(width="1100px", height="520px",theme=ThemeType.DARK))
    .add_xaxis(x)
    .add_yaxis("招生数", y1, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    .add_yaxis("在校生数", y2, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    .add_yaxis("毕业生数", y3, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    .add_yaxis("获得职业资格证书毕业生数", y4, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    .add_yaxis("教职工数", y5, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    .add_yaxis("学校数", y6, is_smooth=True,
#                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average", name="平均值")])
              )
    
    .set_global_opts(title_opts=opts.TitleOpts(title="中职教育发展可视化"),
                        tooltip_opts=opts.TooltipOpts(trigger="axis"),
                        toolbox_opts=opts.ToolboxOpts(is_show=True),
                        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
                        datazoom_opts=opts.DataZoomOpts(range_start=0, range_end=80)
                    )
)
c.render_notebook()
# c.render('中职教育发展总体性数据可视化.html')

#### 现象描述

1 描述性统计分析：

- `招生数`在2009-2010年间到达了顶峰，随后开始逐年下降，及至2014年下降趋势逐渐不明显，直至2018年开始小幅回弹。
- `在校生数`在2010年到达了顶峰，随后开始逐年下降，及至2014年出现了下降拐点，衰减率逐渐下降，直至2019年回弹。
- `毕业生数`及`获得职业资格证书毕业生数`在2013年到达了顶峰，随后开始逐年下降。
- `教职工数量`自2008年开始持续下降，同样及至2013年出现了下降拐点，衰减率逐渐下降，至2018年开始有限的回升。
- `学校数`在2010年到达了顶峰，随后开始逐年下降，直至2019年回弹，但是2020年其数量又突然下降。

2 特征关联性分析：

- 从`时间特征`的角度出发。中职教育发展自2008年以来,实际上是高开低走.与`其它特征`总体上呈`负相关`

- 从`招生数特征`的角度出发。其与`在校生数及教职工数`呈较为显著的`正相关`。
- 从`在校生数特征`的角度出发。其与`招生数,学校数及教职工数`呈较为显著的`正相关`。

- 从`毕业生数特征`的角度出发。其与除时间特征以外的其它特征，都没有显著的相关性。
- 从`获得职业资格证书毕业生数特征`的角度出发。其与`毕业生数`呈较为显著的`正相关`。

- 从`学校数特征`的角度出发。其与`在校生数`及`教职工数`呈较为显著的`正相关`。
- 从`教职工数特征`的角度出发。其与`在校生数,招生数,学校数`呈较为显著的`正相关`。

### 中职教育学科发展分析
- `聚焦于遗失的8年：2010年-2014年：衰退期 挣扎期`

#### 数据可视化

In [80]:
data4.head()

,年份,农林牧渔类,资源与环境类,能源与新能源类,土木水利工程类,加工制造类,石油化工类,轻纺食品类,交通运输类,信息技术类,医药卫生类,休闲保健类,财经商贸类,旅游服务类,文化艺术类,体育与健身,教育类,司法服务类,公共管理与服务类
0,2010,194.5251,11.7013,9.9929,51.6368,341.0804,15.0680,32.0846,92.2995,375.7587,168.3865,7.1353,193.7373,75.8065,94.1092,12.0897,85.9991,8.2666,26.4664
1,2011,225.9595,10.8134,9.0926,58.9347,298.9952,12.8352,21.1447,101.1086,342.0989,165.0724,8.9743,186.8180,73.1429,84.1541,12.3992,111.7272,7.9001,22.4445
2,2012,218.8579,10.8265,8.0738,61.1926,265.8500,11.9058,18.7715,108.3744,297.7614,153.9531,8.4024,184.1117,72.9556,79.4437,12.8165,131.9091,7.0268,19.6907
3,2013,172.2323,9.0803,6.8184,62.4010,230.6826,10.7439,15.2038,113.5676,259.0293,147.0917,8.1875,167.3386,68.9918,74.8355,12.1871,139.6498,6.2428,18.0621
4,2014,132.3974,6.8286,5.7011,61.3386,204.1623,9.0018,12.6192,123.0956,229.1937,146.5838,8.1890,157.0698,67.3944,69.6562,11.8798,136.7044,5.3771,14.6339


In [81]:
data4.columns

Index(['年份', '农林牧渔类', '资源与环境类', '能源与新能源类', '土木水利工程类', '加工制造类', '石油化工类',
       '轻纺食品类', '交通运输类', '信息技术类', '医药卫生类', '休闲保健类', '财经商贸类', '旅游服务类', '文化艺术类',
       '体育与健身', '教育类', '司法服务类', '公共管理与服务类'],
      dtype='object')

In [7]:
def show_pie(data):
    tl = Timeline()
    count = 0

    for i in range(2010, 2015):
        tmp = {}
        x = data.columns.tolist()[1:]
        y = [y for y in dict(data.iloc[count,]).values()][1:]
        for a,b in zip(x,y):
             tmp[a]=b
        tmp1 = sorted(tmp.items(), key=lambda x: x[1], reverse=True)[:8]
        pie = (
            Pie(init_opts=opts.InitOpts(width="1600px", height="800px", bg_color="#D1E9E9"))
            .add("",tmp1,rosetype="radius",radius=["30%", "55%"])
            .set_series_opts(label_opts=opts.LabelOpts(position='outsiede',formatter="{b}:{c}  {d}%"))
        )
        count += 1
        tl.add(pie, "{}年".format(i))
        tl.add_schema(is_auto_play=True,is_rewind_play=False)
    return tl

In [8]:
tab = Tab()
tab.add(show_pie(data4),'分科类在校生数')
tab.add(show_pie(data5),'分科类招生数')
tab.add(show_pie(data6),'分科类毕业生数数')
tab.add(show_pie(data7),'分科类得职业资格证书毕业生数')
tab.render_notebook()
# tab.render('中职教育学科发展数据可视化.html')

#### 现象描述

- `2010年 衰退期伊始`
    - 2010年分学科在校生数排名为：信息技术类（24%） 加工制造类（22%） 农林牧渔类（13%） 财经商贸类（13%） 医药卫生类（11%）
    - 2010年分学科招生数排名为：信息技术类（23%） 加工制造类（19%） 农林牧渔类（18%） 财经商贸类（12%） 医药卫生类（10%）
    - 2010年分学科毕业生数排名为：信息技术类（28%） 加工制造类（28%） 财经商贸类（13%） 医药卫生类（9%）
    - 2010年分学科职业资格证书毕业生数排名为：加工制造类（31%） 信息技术类（30%） 财经商贸类（13%）
- `2014年 挣扎期`
    - 2014年分学科在校生数排名为：信息技术类（19%） 加工制造类（17%） 财经商贸类（13%） 医药卫生类（12%） 教育类（11%） 
    - 2014年分学科招生数排名为：信息技术类（19%） 加工制造类（17%） 财经商贸类（13%） 交通运输类（12%） 医药卫生类（11%） 
    - 2014年分学科毕业生数排名为：信息技术类（21%） 加工制造类（18%） 农林牧渔类（15%） 财经商贸类（13%）
    - 2014年分学科职业资格证书毕业生数排名为：信息技术类（22%） 加工制造类（20%） 农林牧渔类（13%） 财经商贸类（12%） 教育类（11%） 

### 中职教育分省发展分析

#### 数据可视化

In [38]:
data2.shape

(372, 10)

In [37]:
data2.columns

Index(['id', '省份', '年份', '招生数(万人)', '在校学生数(万人)', '毕业生数(万人)',
       '获得职业资格证书毕业生数(万人)', '预计毕业生数(万人)', '教职工总数(万人)', '专任教师数(万人)'],
      dtype='object')

In [36]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                372 non-null    int64  
 1   省份                372 non-null    object 
 2   年份                372 non-null    int64  
 3   招生数(万人)           372 non-null    float64
 4   在校学生数(万人)         372 non-null    float64
 5   毕业生数(万人)          372 non-null    float64
 6   获得职业资格证书毕业生数(万人)  372 non-null    float64
 7   预计毕业生数(万人)        372 non-null    float64
 8   教职工总数(万人)         372 non-null    float64
 9   专任教师数(万人)         372 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 29.2+ KB


In [34]:
data2.describe()

,id,年份,招生数(万人),在校学生数(万人),毕业生数(万人),获得职业资格证书毕业生数(万人),预计毕业生数(万人),教职工总数(万人),专任教师数(万人)
count,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000,372.000000
mean,186.500000,2013.500000,17.850816,48.379587,15.607178,11.028358,15.865778,2.847199,2.138949
std,107.531391,3.456702,13.874437,36.211676,11.921417,8.824027,12.118256,1.906767,1.464435
min,1.000000,2008.000000,0.521900,1.579600,0.243600,0.050900,0.431500,0.073100,0.054100
25%,93.750000,2010.750000,6.139400,18.762025,6.041200,3.804450,6.258950,1.518450,1.037450
50%,186.500000,2013.500000,14.250300,38.198400,12.209950,9.093200,12.171350,2.522750,1.825950
75%,279.250000,2016.250000,25.416550,68.654950,22.469200,16.818750,22.368650,3.834125,2.923950
max,372.000000,2019.000000,74.132000,163.755000,52.902500,42.784200,56.090200,8.429000,6.038900


In [39]:
data2.head()

,id,省份,年份,招生数(万人),在校学生数(万人),毕业生数(万人),获得职业资格证书毕业生数(万人),预计毕业生数(万人),教职工总数(万人),专任教师数(万人)
0,1,新疆维吾尔自治区,2008,9.1979,21.1001,4.2831,1.9845,6.3134,1.4741,0.9933
1,2,新疆维吾尔自治区,2009,8.8484,21.7555,5.9414,2.9639,6.5806,1.5334,1.0504
2,3,新疆维吾尔自治区,2010,9.8894,23.3319,6.6383,3.4953,6.9700,1.4988,1.0084
3,4,新疆维吾尔自治区,2011,9.0177,24.3039,6.4508,3.8080,7.2113,1.4936,1.0382
4,5,新疆维吾尔自治区,2012,8.6751,23.5277,7.1288,4.0546,7.7522,1.4739,1.0352


In [9]:
def show_map(column,title,maxp):
    data21 = data2[data2['年份'].isin([2019])]
    x = [i[:2] for i in data21['省份']]
    x[x.index('内蒙')] = '内蒙古'
    x[x.index('黑龙')] = '黑龙江'

    y = [round(i) for i in data2[column]]##数据！！！！
    y_2 = []
    for i in range(0,31):
        y_2.append(list(y[0:12]))
        y = y[12:]
    y_3 = np.array(y_2).T.tolist()

    tm = Timeline()
    count = 0
    for i in range(2008,2020):
        map = (
            Map(init_opts=opts.InitOpts(bg_color="dimgray", theme=ThemeType.DARK))
            .add("", [list(z) for z in zip(x, y_3[count])], "china")
            .set_global_opts(
                title_opts=opts.TitleOpts(title="中职教育{}年{}热点地图".format(i,title)),
                visualmap_opts=opts.VisualMapOpts(min_=0,max_=maxp,is_piecewise=True, split_number=10)
            )
        )
        count += 1
        tm.add(map, "{}年".format(i))
        tm.add_schema(is_auto_play=True,is_rewind_play=False)
    return tm

tab2 = Tab()
tab2.add(show_map('招生数(万人)','招生数',75),'招生（万人）')
tab2.add(show_map('在校学生数(万人)','在校生数',165),'在校生')
tab2.add(show_map('毕业生数(万人)','毕业生数',53),'毕业生')
tab2.add(show_map('获得职业资格证书毕业生数(万人)','获得职业资格证书毕业生数',43),'获得职业资格证书毕业生')
tab2.add(show_map('教职工总数(万人)','教职工总数',9),'教职工')
tab2.render_notebook()
#tab2.render('中职教育分省发展数据可视化.html')

#### 挖掘性数据分析
- 聚焦于2010年、2014年、2018年这三个时间节点。

##### 分省指标排名

`1 最后的辉煌2010年`

In [15]:
def show_top(year,column):
    a = data2[data2['年份'].isin([year])].sort_values(column,ascending=False)
    return pd.DataFrame(a,columns=['省份', column]).iloc[0:6,:].reset_index(drop=True)

In [16]:
show_top(2010,'招生数(万人)')

,省份,招生数(万人)
0,广东省,74.1320
1,河南省,62.5601
2,四川省,51.1629
3,山东省,42.6954
4,河北省,40.9522
5,广西壮族自治区,38.0908


In [153]:
show_top(2010,'在校学生数(万人)')

,省份,在校学生数(万人)
0,河南省,163.600
1,广东省,154.779
2,四川省,123.694
3,山东省,113.162
4,河北省,112.052
5,江苏省,102.038


In [154]:
show_top(2010,'毕业生数(万人)')

,省份,毕业生数(万人)
0,河南省,52.6333
1,山东省,43.9337
2,河北省,34.8186
3,湖北省,33.5776
4,广东省,33.1741
5,四川省,31.4065


In [155]:
show_top(2010,'获得职业资格证书毕业生数(万人)')

,省份,获得职业资格证书毕业生数(万人)
0,河南省,29.6116
1,山东省,29.5311
2,广东省,23.7823
3,江苏省,22.0945
4,四川省,21.6459
5,湖北省,20.8282


In [156]:
show_top(2010,'教职工总数(万人)')

,省份,教职工总数(万人)
0,河南省,8.3165
1,山东省,7.8769
2,河北省,6.7544
3,广东省,5.8754
4,江苏省,5.7667
5,四川省,5.0376


`2 遗失的8年:挣扎期：2014年`

In [157]:
show_top(2014,'招生数(万人)')

,省份,招生数(万人)
0,四川省,44.3842
1,广东省,41.7047
2,河南省,39.3380
3,安徽省,33.4170
4,山东省,31.9143
5,广西壮族自治区,27.1153


In [158]:
show_top(2014,'在校学生数(万人)')

,省份,在校学生数(万人)
0,广东省,128.2210
1,河南省,110.3860
2,四川省,107.9230
3,山东省,94.8167
4,安徽省,91.4742
5,广西壮族自治区,78.2675


In [159]:
show_top(2014,'毕业生数(万人)')

,省份,毕业生数(万人)
0,四川省,46.8087
1,广东省,45.7010
2,河南省,41.9551
3,山东省,35.4032
4,安徽省,32.6438
5,河北省,29.7687


In [160]:
show_top(2014,'获得职业资格证书毕业生数(万人)')

,省份,获得职业资格证书毕业生数(万人)
0,四川省,42.7842
1,河南省,32.2300
2,安徽省,29.7133
3,山东省,28.4896
4,广东省,26.1493
5,江苏省,23.0586


In [161]:
show_top(2014,'教职工总数(万人)')

,省份,教职工总数(万人)
0,河南省,6.7699
1,山东省,6.4488
2,广东省,5.8051
3,河北省,5.7500
4,江苏省,5.2976
5,四川省,5.2400


`3 重整旗鼓：2018年`

In [162]:
show_top(2018,'招生数(万人)')

,省份,招生数(万人)
0,河南省,39.1987
1,四川省,32.5773
2,广东省,29.7190
3,安徽省,29.4300
4,河北省,27.5767
5,广西壮族自治区,24.7944


In [163]:
show_top(2018,'在校学生数(万人)')

,省份,在校学生数(万人)
0,河南省,110.1600
1,广东省,86.7254
2,四川省,82.0060
3,安徽省,75.2810
4,山东省,75.0142
5,河北省,72.4282


In [164]:
show_top(2018,'毕业生数(万人)')

,省份,毕业生数(万人)
0,四川省,32.7834
1,广东省,31.8470
2,河南省,31.4036
3,安徽省,25.7212
4,山东省,25.0210
5,河北省,23.0532


In [17]:
show_top(2018,'获得职业资格证书毕业生数(万人)')

,省份,获得职业资格证书毕业生数(万人)
0,四川省,29.2927
1,广东省,23.1492
2,河南省,22.9460
3,安徽省,20.2218
4,山东省,18.0701
5,江苏省,17.9215


In [166]:
show_top(2018,'教职工总数(万人)')

,省份,教职工总数(万人)
0,河南省,6.0030
1,山东省,5.9304
2,河北省,5.9202
3,广东省,5.6750
4,江苏省,5.0137
5,四川省,4.8342
